# test functions to get citations
## you will need the devtools package
install.packages("devtools")

In [ ]:
getwd()
devtools::load_all('/replace/with/path/to/your/clone/of/package/rotl')

In [ ]:
##' @export
##' @param study_ids is a vector of study ids
get_publication.vector <- function(study_ids){
  citations <- sapply(study_ids, function(x) rotl::get_publication.study_meta(rotl::get_study_meta(study_id = x)))
  return(citations)
}

In [ ]:
# res is an output from .tol_induced_subtree or .tol_subtree
# returns a list with tree and study ids supporting a subtree
studies_from_otl <- function(res){
  if (is.list(res)) {
    if (!is.null(res$supporting_studies)) {
        studies <- res$supporting_studies
    }
    else {
        stop("Cannot find supporting studies")
    }
  }
  study_ids <- sapply(stringr::str_split(studies, "@"), '[', 1)
  trees <- sapply(stringr::str_split(studies, "@"), '[', 2)
  return(list(study_ids = study_ids, trees = trees))
}

In [ ]:
# add arguments to .tol_induced_subtree
# get_citations = TRUE

tol_induced_subtree <- function(ott_ids = NULL, node_ids = NULL, label_format = NULL,
                                file, get_citations = TRUE, ...) {
  res <- .tol_induced_subtree(
    ott_ids = ott_ids, node_ids = node_ids,
    label_format = label_format, ...
  )
  if(get_citations){
    if (!missing(file)) {
      filecit <- paste0(file, "_citations.txt")
    } else {
      filecit <- "citations.txt"
    }
    citations <- get_publication.vector(studies_from_otl(res)$study_ids)
    unlink(filecit)
    cat(paste(citations, collapse = '\n\n'), file = filecit)
  }
  if (!missing(file)) {
    unlink(file)
    cat(res$newick, file = file)
    return(file.exists(file))
  } else {
    phy <- phylo_from_otl(res)
    return(phy)
  }
}

In [ ]:
# try the new function when NO file name is provided
tol_induced_subtree(c(292466, 267845, 316878, 102710), get_citations = TRUE)

In [ ]:
# try the new function when the file name IS provided
tol_induced_subtree(c(292466, 267845, 316878, 102710), file = "test", get_citations = TRUE)